## Parsing Micron Sonar File from CSV format

In [276]:
# import external libraries
import csv
import datetime
import dateutil
import importlib
import pandas as pd
import math
import numpy as np
import os
import seaborn as sns 
import scipy
import sys
from matplotlib import pyplot as plt 

# add parent directory to the path for importing modules 
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import micron_reader
import MicronEnsemble
import MicronTimeSeries

# use Seaborn settings for plotting 
sns.set()

## Plotting Function

In [407]:
def plot_ensemble(ensemble, location, output_file):
    
    # opacity of the peak 
    if ensemble.peak_width_bin == 0: 
        peak_alpha = 0
    else: 
        peak_alpha = 0.3
    
    months = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr',  5:'May',  6:'Jun',
              7:'Jul', 8:'Aug', 9:'Sep',10:'Oct', 11:'Nov', 12:'Dec'}
    deg_in_quadrant = 90

    sns.set(font_scale = 1.5)
    fig, ax = plt.subplots(figsize=(15,8))
    ax.plot(ensemble.intensity_data, linewidth=3, color='tab:blue')
    plt.axvspan(ensemble.peak_start_bin, ensemble.peak_end_bin, alpha=peak_alpha, color='tab:purple')

    fig.suptitle('Micron Sonar Ensemble, %s, %i %s %i' % 
                 (location, ensemble.day, months[ensemble.month], ensemble.year), 
                 fontsize=22, fontweight='bold')

    ax.set_title("Incidence: %3i$^\circ$   Bearing: %3i$^\circ$   Intensity: %2d dB   Peak Width: %.2f m" % 
                   (abs(ensemble.bearing - deg_in_quadrant), 
                    ensemble.bearing,
                    ensemble.max_intensity,
                    ensemble.peak_width), 
                 fontsize=18,
                 fontname='Courier New')

    num_ticks = 11
    xticks = np.arange(0, ensemble.dbytes, ensemble.dbytes/(num_ticks-1))
    xticks = np.append(xticks, ensemble.dbytes)
    xtick_labels = ["%.1f" % (e*ensemble.bin_size) for e in xticks]
    ax.set_ylim(0,25) # TODO need to set y-limit better 
    ax.set_xticks(xticks)
    ax.set_xticklabels(xtick_labels)
    ax.set_xlabel('Distance from Transducer [m]')
    ax.set_ylabel('Intensity [dB]')
    ax.legend(['Intensity', 'Rolling Median','FWHM'], loc='best')
    plt.savefig("../figs/%s.png" % (output_file))
    plt.close()

## Reload Modules after Making Adjustments to the Code

In [409]:
importlib.reload(MicronEnsemble)
importlib.reload(MicronTimeSeries)
importlib.reload(micron_reader)

filepath = "/Users/zduguid/Dropbox (MIT)/MIT-WHOI/NSF Arctic NNA/Research Activities/2020-01-24_WHOI-Tank-Sonar-Testing/csv/Fri_24_Jan_13_34.CSV"
location = "Woods Hole MA"
date     = (2020, 1, 24)

ensemble = micron_reader.micron_reader(filepath, location, date)
output   = "first-ensemble.png"
plot_ensemble(ensemble, location, output)
# + see if plotting function below can be made to work with new class

In [402]:
"""
    # store current angle and rounded angle 
    angle_current         = data_array[data_lookup['Bearing']]
    angle_current_rounded = math.floor(angle_current/angle_increment)*angle_increment
    
    # TODO make this a sub-method
    # compute the change in angle since a plot has been generated 
    if (not angle_previous): 
        angle_delta = 0
    else:  
        # add change current angle difference to the running sum since last plot 
        if angle_current - angle_previous > 0:
            angle_delta += angle_current - angle_previous 
        # account for instance where 
        else: 
            angle_delta += angle_current + deg_in_circle - angle_previous 
    
    # update previous angle 
    angle_previous = angle_current 
    
    # generate plot for every 20 degrees, avoid replotting area already plotted
    if ((angle_current_rounded % angle_increment == 0) and 
        (angle_delta > angle_increment/2) and 
        (angle_current_rounded not in angles_plotted) and 
        (plot_on)):
"""

"\n    # store current angle and rounded angle \n    angle_current         = data_array[data_lookup['Bearing']]\n    angle_current_rounded = math.floor(angle_current/angle_increment)*angle_increment\n    \n    # TODO make this a sub-method\n    # compute the change in angle since a plot has been generated \n    if (not angle_previous): \n        angle_delta = 0\n    else:  \n        # add change current angle difference to the running sum since last plot \n        if angle_current - angle_previous > 0:\n            angle_delta += angle_current - angle_previous \n        # account for instance where \n        else: \n            angle_delta += angle_current + deg_in_circle - angle_previous \n    \n    # update previous angle \n    angle_previous = angle_current \n    \n    # generate plot for every 20 degrees, avoid replotting area already plotted\n    if ((angle_current_rounded % angle_increment == 0) and \n        (angle_delta > angle_increment/2) and \n        (angle_current_rounded 

## Print All Attributes of the Micron Sonar Ensemble 

In [397]:
print("ENSEMBLE (len: %i)" % ensemble.ensemble_size)
print("HEADER (len: %i)" % ensemble.header_len)
print(ensemble.line_header)
print(ensemble.date_time)
print(ensemble.node)
print(ensemble.status)
print(ensemble.hdctrl)
print(ensemble.range_scale)
print(ensemble.gain)
print(ensemble.slope)
print(ensemble.ad_low)
print(ensemble.ad_span)
print(ensemble.left_lim)
print(ensemble.right_lim)
print(ensemble.steps)
print(ensemble.bearing)
print(ensemble.dbytes)
print("DERIVED (len: %i)" % ensemble.derived_len)
print(ensemble.bin_size)
print(ensemble.max_intensity)
print(ensemble.max_intensity_bin)
print(ensemble.peak_start)
print(ensemble.peak_start_dist)
print(ensemble.peak_end)
print(ensemble.peak_end_dist)
print(ensemble.peak_width)
print(ensemble.peak_width_dist)
print("INTENSITIES (len: %i)" % len(ensemble.intensity_data))

ENSEMBLE (len: 341)
HEADER (len: 15)
1
1579908848.0
2
16
8965
2.0
2
100
0.9411764705882353
23.84313725490196
30.037500000000023
149.9625
0.45
30.26249999999999
317
DERIVED (len: 9)
0.006309148264984227
12.862745098039216
152
140
0.8832807570977917
158
0.9968454258675079
18
0.11356466876971609
INTENSITIES (len: 317)


## Playing with Bit-Encoding of Micron Data

In [398]:
int("8923", 16)
int("16", 16)
bin(int("16", 16))
bin(8923)

bin(16971)[2:][::-1]
int('1101001001000010'[::-1],2)

# 8923 hex -> 0b   1000 1001 0010 0011
# 8923 int -> 0b   0010 0010 1101 1011'

# 8967 hex -> 0b   1000 1001 0110 0111
# 8967 int -> 0b   0010 0011 0000 0111'

# 144  hex -> 0b 1 0100 0100
# 144  int -> 0b   1001 0000
# 16   hex -> 0b   0001 0110

# print(header[14], row1[14]) # column 14 is the number of data points in the scanline 
# print(header[15], row1[15]) # column 15 is the first value of an array DBytes long

16971